PostgreSQL의 확장기능인 PGVector의 사용법에 대해 정리한 문서입니다.

## PG psql commands

In [ ]:
\l -- 현재 클러스터의 데이터베이스 목록 표시
\c [database_name] -- 특정 데이터베이스로 연결
\conninfo -- 현재 연결된 데이터베이스 정보

\du -- 사용자 역할
\dt -- 테이블 확인
\dt schema_name.* -- 특정 스키마 테이블 확인
\d table_name -- 테이블 구조 확인
\dv -- 뷰 목록 확인
\di -- 인덱스 목록 확인
\ds -- 시퀀스 목록 확인
\df+ [function_name]: -- 특정 함수의 상세 정보 확인

\dx -- 현재 설치된 확장 확인
\f -- 함수(Function) 목록 확인
\timing on 
\?
\q

## pgvector basic

pgvector 시작하기

In [ ]:
CREATE EXTENSION vector;
CREATE EXTENSION IF NOT EXISTS vector;
SELECT * FROM pg_extension; -- 현재 존재하는 extension 확인 = \dx
DROP TABLE vectors;

CREATE TABLE items (id bigserial PRIMARY KEY, embedding vector(3));
INSERT INTO items (embedding) VALUES ('[1,2,3]'), ('[4,5,6]');

SELECT * from items;
SELECT * FROM items ORDER BY embedding <-> '[3,1,2]' LIMIT 5;

Storing

In [ ]:
SELECT * FROM items ORDER BY embedding <-> '[3,1,2]' LIMIT 5;

ALTER TABLE items ADD COLUMN embedding vector(3);

INSERT INTO items (embedding) VALUES ('[1,2,3]'), ('[4,5,6]');

COPY items (embedding) FROM STDIN WITH (FORMAT BINARY);

INSERT INTO items (id, embedding) VALUES (1, '[1,2,3]'), (2, '[4,5,6]')
    ON CONFLICT (id) DO UPDATE SET embedding = EXCLUDED.embedding;

UPDATE items SET embedding = '[1,2,3]' WHERE id = 1;

DELETE FROM items WHERE id = 1;

Querying

In [ ]:
SELECT * FROM items ORDER BY embedding <-> '[3,1,2]' LIMIT 5; 

SELECT
	id, 
	embedding,
	TRUNC((embedding <-> '[3,1,2]')::numeric,2) as L2 
FROM items 
ORDER BY L2 
LIMIT 5;

SELECT
	id, 
	embedding,
	TRUNC((embedding <#> '[1,1,3]')::numeric,2) as L2 
FROM items 
ORDER BY L2 
LIMIT 5;

Distances


* <-> - L2 distance
* <#> - (negative) inner product
* <=> - cosine distance
* <+> - L1 distance
* <~> - Hamming distance : binary 타입에 대해 지원
* <%> - Jaccard distance : binary 타입에 대해 지원

In [ ]:
SELECT '[1,2,3]'::vector <=> '[3,1,2]'::vector AS distance;

Aggregates

In [ ]:
SELECT AVG(embedding) FROM items;
SELECT category_id, AVG(embedding) FROM items GROUP BY category_id;

## Indexing

인덱스를 추가하여 근사-nn 검색을 할 수 있음. 속도를 위해 recall을 약간 희생. \
두가지 방식 HNSW vs IVFFlat - HNSW가 구축에는 시간이 더 오래 걸리고 메모리도 많이 차지하지만 검색은 빠름.


### 1. HNSW

* vector - up to 2,000 dimensions
* halfvec - up to 4,000 dimensions (added in 0.7.0)
* bit - up to 64,000 dimensions (added in 0.7.0)
* sparsevec - up to 1,000 non-zero elements (added in 0.7.0)

In [ ]:
CREATE INDEX ON items USING hnsw (embedding vector_l2_ops);
CREATE INDEX ON items USING hnsw (embedding vector_ip_ops);
CREATE INDEX ON items USING hnsw (embedding vector_cosine_ops);
CREATE INDEX ON items USING hnsw (embedding vector_l1_ops);

HNSW 사용시, 인덱스 생성에 적용하는 옵션과 쿼리시에 적용하는 옵션이 있음.

생성시에 적용하는 옵션으로 알고리즘에 관련된 m과 ef_construction가 있음. 둘 다 높을 수록 정확도가 증가하지만, 메모리 사용량이 커짐.
- m - 각 레이어별로 최대 연결 가능 숫자(default=16)
- ef_construction - 인덱스 생성시 고려할 후보 개수

In [ ]:
CREATE INDEX ON items USING hnsw (embedding vector_l2_ops) WITH (m = 16, ef_construction = 64);

연산자원 할당과 관련해서는 maintenance_work_mem과 max_parallel_workers를 설정할 수 있음.

In [ ]:
SET maintenance_work_mem = '8GB'; # 너무 높게 잡지 말 것
SET max_parallel_workers = 7; # 8이 기본값

쿼리시에 적용하는 알고리즘과 관련된 옵션으로 hnsw.ef_search가 있고, 트랙잭션 안에 넣어주는 것이 좋다.

In [ ]:
BEGIN;
SET hnsw.ef_search = 100;
SELECT ...
END;

pg_stat_progress_create_index을 통해 진행상황을 확인할 수 있다.

In [ ]:
SELECT
    phase, 
    round(100.0 * blocks_done / nullif(blocks_total, 0), 1) AS "%" 
FROM pg_stat_progress_create_index; 

반복진행 시 다음을 조정한다

In [ ]:
SET hnsw.max_scan_tuples = 20000;
SET hnsw.scan_mem_multiplier = 2;

### 2. IVFFlat

IVFFlat은 데이터포인트를 여러 리스트(클러스터)로 분할하고, 쿼리 벡터와 가까운 클러스터를 찾는 방식으로 수행된다. \
더 빠르고 메모리도 적게 차지하지만, 쿼리 퍼포먼스는 떨어진다. 데이터타입은 vector, halfvec, bit가 지원된다.

1) 테이블에 어느정도 데이터가 있을 때 인덱스를 생성할 것
2) 적절한 list의 수를 선택할 것. 1M 행 이하에서는 rows/1000, 이상에서는 sqrt(rows)에서 시작하라.
3) 적절한 probes 수를 선택할 것. 높을 수록 재현율은 높아진다. sqrt(lists)에서 시작할 것. 

In [ ]:
CREATE INDEX ON items USING ivfflat (embedding vector_l2_ops) WITH (lists = 100);
CREATE INDEX ON items USING ivfflat (embedding vector_ip_ops) WITH (lists = 100);
CREATE INDEX ON items USING ivfflat (embedding vector_cosine_ops) WITH (lists = 100);
CREATE INDEX ON items USING ivfflat (embedding bit_hamming_ops) WITH (lists = 100);

BEGIN;
SET LOCAL ivfflat.probes = 10;
SET max_parallel_maintenance_workers = 7; -- 2가 기본값
SET max_parallel_workers = 7; -- 설정 가능
SELECT ...
COMMIT;

SELECT phase, round(100.0 * tuples_done / nullif(tuples_total, 0), 1) AS "%" FROM pg_stat_progress_create_index; -- 진행상황 확인

반복진행 시 다음을 조정

In [ ]:
SET ivfflat.max_probes = 100;

## pgVector의 벡터 자료형식

pgvector는 용도에 맞는 다양한 데이터 형식을 지원한다.

* vector - up to 2,000 dimensions
* halfvec - up to 4,000 dimensions (added in 0.7.0)
* bit - up to 64,000 dimensions (added in 0.7.0)
* sparsevec - up to 1,000 non-zero elements (added in 0.7.0)

**Vector**

벡터는 float4로 저장됨. 총크기는 4*dims+8 바이트. 약 7자리 소수점 정확도(123.4567).

In [ ]:
INSERT INTO items (embedding) VALUES('[0,0,123.45678901234]');
select * from items; -- 123.4567까지만 출력된다

연산자에 대해 알아보자.

- \+ 원소별 덧셈 Element-wise addition
- \- 원소별 뺄셈 Element-wise subtraction
- \* 원소별 곱셈 Element-wise product(0.5.0)
- || 이어붙이기 Concatenate(0.7.0)
- <-> L2거리(유클리드거리) L2 distance
- <#> 내적거리 Negative Inner Product
- <=> 코사인거리 Cosine Distance
- <+> L1거리(택시거리,맨하탄거리) L1 Distance(0.7.0)

In [ ]:
SELECT '[1,2,3]'::vector + '[3,1,2]'::vector AS RESULT;
SELECT '[1,2,3]'::vector - '[3,1,2]'::vector AS RESULT;

SELECT '[1,2,3]'::vector <-> '[3,1,2]'::vector AS RESULT;
SELECT '[1,2,3]'::vector <#> '[3,1,2]'::vector AS RESULT;
SELECT '[1,2,3]'::vector <=> '[3,1,2]'::vector AS RESULT;
SELECT '[1,2,3]'::vector <+> '[3,1,2]'::vector AS RESULT;

SELECT l2_distance('[1,2,3]'::vector, '[3,1,2]'::vector) AS RESULT;
SELECT '[1,2,3]'::vector <-> '[3,1,2]'::vector AS RESULT;

SELECT cosine_distance('[1,2,3]'::vector, '[3,1,2]'::vector) AS RESULT;
SELECT '[1,2,3]'::vector <=> '[3,1,2]'::vector AS RESULT

SELECT -inner_product('[1,2,3]'::vector, '[3,1,2]'::vector) AS RESULT;
SELECT '[1,2,3]'::vector <#> '[3,1,2]'::vector AS RESULT;

SELECT vector_dims('[1,2,3]'::vector) AS DIMS; -- 차원
SELECT vector_norm('[1,2,3]'::vector) AS DIMS; -- 크기
SELECT subvector('[1,2,3]'::vector, 2, 3) AS SUB_VECTOR; -- 2가 시작지점, 3까지 포함한 종료지점. end_point가 실제 크기를 넘어가도 에러가 발생하지는 않음.
SELECT binary_quantize('[0,-1,2]'::vector) AS BINARY; -- 0을 초과하는 값만 1로 리코딩

In [ ]:
SELECT AVG(embedding) AS AVG FROM items; -- 집계함수
SELECT SUM(embedding) AS SUM FROM items; -- 집계함수

**Halfvec Type**

하프벡터는 float2로 저장됨. 총크기는 2*dims+8 바이트. 약 3자리 소수점 정확도(123.4567).

다른 것은 거의 동일하다.

**Bit Type**

dimensions / 8 + 8 바이트로 저장됨.

격자기반 거리함수를 적용할 수 있다.

In [ ]:
SELECT hamming_distance('01100101'::bit(8), '01000001'::bit(8)) AS RESULT; 
SELECT jaccard_distance('00101'::bit(5), '01001'::bit(5)) AS RESULT; # 교집합 1개, 합집합 3개

**Sparsevec**

In [ ]:
SELECT l2_distance('{1:1,3:2,5:3}/5'::sparsevec, '{1:3,2:1,5:3}/5'::sparsevec) AS RESULT;

### 다양한 형식으로 데이터를 INSERT, INDEXING, QUERY하는 예문들

In [ ]:
CREATE TABLE items (id bigserial PRIMARY KEY, embedding halfvec(3));

CREATE INDEX ON items USING hnsw ((embedding::halfvec(3)) halfvec_l2_ops);

SELECT * FROM items ORDER BY embedding::halfvec(3) <-> '[1,2,3]' LIMIT 5;

In [ ]:
CREATE TABLE items (id bigserial PRIMARY KEY, embedding bit(3));

INSERT INTO items (embedding) VALUES ('000'), ('111');

SELECT * FROM items ORDER BY embedding <~> '101' LIMIT 5;

In [ ]:
CREATE INDEX ON items USING hnsw ((binary_quantize(embedding)::bit(3)) bit_hamming_ops);

SELECT * FROM items ORDER BY binary_quantize(embedding)::bit(3) <~> binary_quantize('[1,-2,3]') LIMIT 5;

SELECT * FROM (
    SELECT * FROM items ORDER BY binary_quantize(embedding)::bit(3) <~> binary_quantize('[1,-2,3]') LIMIT 20
) ORDER BY embedding <=> '[1,-2,3]' LIMIT 5; # Re-rank by the original vectors for better recall

In [ ]:
CREATE TABLE items (id bigserial PRIMARY KEY, embedding sparsevec(5));

INSERT INTO items (embedding) VALUES ('{1:1,3:2,5:3}/5'), ('{1:4,3:5,5:6}/5');

SELECT * FROM items ORDER BY embedding <-> '{1:3,3:1,5:2}/5' LIMIT 5;

In [ ]:
-- Postgres full-text search와 함께 사용할 것
-- You can use Reciprocal Rank Fusion or a cross-encoder to combine results.
SELECT id, content FROM items, plainto_tsquery('hello search') query
    WHERE textsearch @@ query ORDER BY ts_rank_cd(textsearch, query) DESC LIMIT 5;

# 정리중인 내용

### 환경설정

In [ ]:
-- shared_buffers 설정 확인
-- shared_buffers는 PostgreSQL이 데이터베이스 페이지를 캐시하는 데 사용하는 메모리 양을 나타냄.
-- 일반적으로 서버 메모리의 약 1/4 정도로 설정하는 것이 좋음.
SHOW shared_buffers;

-- work_mem 설정 확인
-- work_mem은 정렬, 해시 조인 등 임시 데이터를 처리할 때 사용하는 메모리의 크기를 나타냄.
-- 쿼리의 복잡성에 따라 적절히 조정. 기본적으로 4MB로 설정됨.
SHOW work_mem;

-- maintenance_work_mem 설정 확인
-- maintenance_work_mem은 VACUUM, CREATE INDEX 등 유지 관리 작업 시 사용하는 메모리의 크기.
-- 일반적으로 기본값보다 크게 설정하면 유지 관리 작업 속도가 향상될 수 있음.
SHOW maintenance_work_mem;

-- effective_cache_size 설정 확인
-- effective_cache_size는 운영 체제에서 사용할 수 있는 파일 시스템 캐시 크기를 PostgreSQL에 알려주는 값.
-- 이 값을 통해 쿼리 플래너가 캐시를 활용한 쿼리 성능을 최적화함.
SHOW effective_cache_size;

-- shared_buffers 값 변경
-- shared_buffers 값을 64MB로 설정. 이 값은 서버의 메모리 크기에 따라 적절히 조정해야 함.
ALTER SYSTEM SET shared_buffers = '64MB';

-- 설정 파일 변경 내용을 PostgreSQL 서버에 적용
-- pg_reload_conf()는 서버를 재시작하지 않고도 설정 파일 변경 사항을 적용함.
-- 그러나 shared_buffers는 서버 재시작이 필요하므로 이 명령으로는 즉시 적용되지 않음.
SELECT pg_reload_conf();

### 성능개선 튜닝

In [ ]:


Tuning
Use a tool like PgTune to set initial values for Postgres server parameters. For instance, shared_buffers should typically be 25% of the server’s memory. You can find the config file with:

SHOW config_file;
And check individual settings with:

SHOW shared_buffers;
Be sure to restart Postgres for changes to take effect.

Loading
Use COPY for bulk loading data (example).

COPY items (embedding) FROM STDIN WITH (FORMAT BINARY);
Add any indexes after loading the initial data for best performance.

Indexing
See index build time for HNSW and IVFFlat.

In production environments, create indexes concurrently to avoid blocking writes.

CREATE INDEX CONCURRENTLY ...
Querying
Use EXPLAIN ANALYZE to debug performance.

EXPLAIN ANALYZE SELECT * FROM items ORDER BY embedding <-> '[3,1,2]' LIMIT 5;
Exact Search
To speed up queries without an index, increase max_parallel_workers_per_gather.

SET max_parallel_workers_per_gather = 4;
If vectors are normalized to length 1 (like OpenAI embeddings), use inner product for best performance.

SELECT * FROM items ORDER BY embedding <#> '[3,1,2]' LIMIT 5;
Approximate Search
To speed up queries with an IVFFlat index, increase the number of inverted lists (at the expense of recall).

CREATE INDEX ON items USING ivfflat (embedding vector_l2_ops) WITH (lists = 1000);
Vacuuming
Vacuuming can take a while for HNSW indexes. Speed it up by reindexing first.

REINDEX INDEX CONCURRENTLY index_name;
VACUUM table_name;

### 모니터링

In [ ]:
# Monitor performance with pg_stat_statements (be sure to add it to shared_preload_libraries).

CREATE EXTENSION pg_stat_statements;

# Get the most time-consuming queries with:

SELECT query, calls, ROUND((total_plan_time + total_exec_time) / calls) AS avg_time_ms,
    ROUND((total_plan_time + total_exec_time) / 60000) AS total_time_min
    FROM pg_stat_statements ORDER BY total_plan_time + total_exec_time DESC LIMIT 20;

# Monitor recall by comparing results from approximate search with exact search.

BEGIN;
SET LOCAL enable_indexscan = off; -- use exact search
SELECT ...
COMMIT;